# Popis řešení

## Použité předpoklady
Příchod každého pacienta je podmíněně nezávislý na příchodů ostatních pacientů. Podmíněno je to zde vnějšími podmínkami, konkrétně jde o:
- čas:
    - den v týdnu,
    - den v roce,
    - hodina dne,
- počasí v Praze:
    - vlhkost vzduchu,
    - venkovní teplota,
    - atmosférický tlak vzduchu,
- Google - trendy mobility v Praze:
    - Retail & Recreation (restaurace, kavárny, kina...),
    - Residential (pohyb v okolí domova),
- Apple - trend mobility v Praze:
    - pouze jednotné vyjádření poměrné mobility,
- Waze v Praze:
    - Change in Driven Miles/KMs (v podstatě stupeň dopravy).

Tyto veličiny slouží jako vstupy modelu. Dále předpokládáme, že frekvence příchodů je v čase proměnlivá, nemění se skokově, ale spojitě.

## Pravděpodobnostní model
Zvolili jsme Poissonovo rozdělení s parametrem $r$, tedy frekvencí příchodů pacientů za jednotku času. Jednotkou času je 1 hodina.
Frekvence příchodů je v čase proměnlivá a modelujeme ji pomocí Gaussovského procesu.
\begin{equation}
r(t) \sim \mathcal{GP}
\end{equation}

In [1]:
import math
import torch
import pyro
import tqdm
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
import matplotlib as mpl
mpl.rc_file_defaults()

In [ ]:
dfactory_online = data_loader.OnlineFactory(data_frequency=hack_config.data_frequency,
                                        teams=hack_config.teams,
                                        hospital=hack_config.hospital,
                                        data_folder=hack_config.data_folder,
                                        dtype=dtype)
ora = Oracle(hack_config.model_path, dsfactory=dfactory_online)
preds = ora.get_arrival_prob_mass(n_arrivals=8,
                                    to_future=2,
                                    to_past=10)
ax = sns.heatmap(preds.T, vmin=0, vmax=1,
    annot=True,
    fmt='.1f',
    cmap="YlGnBu",
    cbar_kws={'label': 'Probability'})
ax.set_xticklabels(preds.index.strftime('%d. %m. %H:%M'))
ax.invert_yaxis()
ax.set_xlabel('Time')
ax.set_ylabel('Number of Arrivals')